In [24]:
"""국토교통부 건축물대장 전유공용면적
- 링크: https://www.data.go.kr/data/15044713/openapi.do
- Count가 어마어마함.. 양천구 + 목동만 30만개
"""
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import io
import xmltodict
import json
from tqdm import tqdm

# import xml.etree.ElementTree as et


# load .env
load_dotenv()
getRTMSDataSvcAptTradeDev_en = os.environ.get("getRTMSDataSvcAptTradeDev_en")
getRTMSDataSvcAptTradeDev_de = os.environ.get("pubkey_de")

In [25]:
SIGUNGU_CODE = 11470  # 시군구코드: 서울특별시 양천구
B_CODE = 10200  # 법정동코드: 목동

In [43]:
url = "http://apis.data.go.kr/1613000/BldRgstService_v2/getBrExposPubuseAreaInfo"

In [44]:
page_list = list(range(1, 31))

In [45]:
dict_aggregation = []
for page_no in tqdm(page_list):
    params = {
        "serviceKey": getRTMSDataSvcAptTradeDev_de,
        "sigunguCd": SIGUNGU_CODE,
        "bjdongCd": B_CODE,
        "numOfRows": 10000,
        "pageNo": page_no
    }

    response = requests.get(url, params=params)
    try:
        dict_content = xmltodict.parse(response.content)["response"]["body"]["items"]["item"]
        dict_aggregation.extend(dict_content)
    except Exception as e:
        print(e)
        print(page_no)

100%|██████████| 30/30 [07:53<00:00, 15.79s/it]


### Error

In [46]:
# get TotalCount
xmltodict.parse(response.content)["response"]["body"]["totalCount"]

'298570'

In [13]:
a = b"\xea\xb2\x8c\xec\x9d\xb4\xed\x8a\xb8\xec\x9b\xa8\xec\x9d\xb4 \xeb\x82\xb4\xeb\xb6\x80 \xec\x84\x9c\xeb\xb9\x84\xec\x8a\xa4 \xec\x98\xa4\xeb\xa5\x98"

In [15]:
a.decode("utf-8")

'게이트웨이 내부 서비스 오류'

In [48]:
len(dict_aggregation)

298570

In [49]:
df = pd.DataFrame(dict_aggregation)
df.to_csv("data/서울특별시_양천구_건출물대장_전유공용면적.csv",
          encoding="utf8",
          index=False
)

In [54]:
df[df["newPlatPlc"] == "서울특별시 양천구 목동로 212"].to_csv("data/서울특별시_양천구(목동신시가지아파트7차)_건출물대장_전유공용면적.csv",
          encoding="utf8",
          index=False
)